In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm

# 1. Define paths for ImageNet validation images and annotations
IMAGE_DIR = "/home/kajm20/mnist/ILSVRC/Data/CLS-LOC/val"  # Path to validation images
ANNOTATION_DIR = "/home/kajm20/mnist/ILSVRC/Annotations/CLS-LOC/val"  # Path to XML annotations

# 2. Define transformations for EfficientNet input (resize, crop, normalize)
imagenet_transform = transforms.Compose([
    transforms.Resize(256),  # Resize the image to 256x256
    transforms.CenterCrop(224),  # Crop the image to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean/std
])

# 3. Load the synset mapping
synset_mapping_path = "/home/kajm20/mnist/ILSVRC/LOC_synset_mapping.txt"
wordnet_to_imagenet = {}

# Load synset mapping from file
with open(synset_mapping_path) as f:
    for idx, line in enumerate(f.readlines()):
        wordnet_id, _ = line.split(' ', 1)  # Get WordNet ID from the line (skip class name)
        wordnet_to_imagenet[wordnet_id] = idx  # Map WordNet ID to class index

# 4. Define the custom dataset class
class ImageNetValDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform

        # Get all annotation file names
        self.annotation_files = sorted(os.listdir(annotation_dir))

    def __len__(self):
        return len(self.annotation_files)

    def __getitem__(self, idx):
        # Get annotation file path
        annotation_path = os.path.join(self.annotation_dir, self.annotation_files[idx])
        
        # Parse XML to extract class label
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        wordnet_id = root.find("object").find("name").text  # WordNet ID, e.g., 'n01751748'

        # Use the synset mapping to convert WordNet ID to ImageNet class index
        class_idx = wordnet_to_imagenet.get(wordnet_id, -1)  # Default to -1 if not found (shouldn't happen)

        # Get image filename from XML and construct image path
        image_filename = root.find("filename").text + ".JPEG"
        image_path = os.path.join(self.image_dir, image_filename)

        # Load and transform image
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, class_idx

# 5. Initialize the dataset and dataloader
imagenet_val_dataset = ImageNetValDataset(IMAGE_DIR, ANNOTATION_DIR, transform=imagenet_transform)
imagenet_val_loader = DataLoader(imagenet_val_dataset, batch_size=32, shuffle=False, num_workers=4)

# 6. Define the model (EfficientNet-B0 with pre-trained weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.efficientnet_b0(weights='DEFAULT')  # Load pre-trained EfficientNet-B0 model
model.to(device)
model.eval()  # Set the model to evaluation mode




EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [7]:
# 7. Define the evaluation function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for inference
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images) 
            _, predicted = torch.max(outputs, 1)  # Get highest probability class
            correct += (predicted == labels).sum().item()  # Count correct predictions
            total += labels.size(0)

    accuracy = (correct / total) * 100
    return accuracy

# 8. Evaluate the model on ImageNet validation set
accuracy = evaluate_model(model, imagenet_val_loader)
print(f"EfficientNet-B0 Top-1 Accuracy on ImageNet: {accuracy:.2f}%")

  0%|          | 0/25000 [00:00<?, ?it/s]

torch.Size([2, 3, 224, 224])
torch.Size([2, 1000])


ZeroDivisionError: division by zero

In [47]:
import os
import torch
import torch.nn as nn
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm
import inspect  # Import inspect to get source code of forward methods

# 1. Define paths for ImageNet validation images and annotations
IMAGE_DIR = "/home/kajm20/mnist/ILSVRC/Data/CLS-LOC/val"  
ANNOTATION_DIR = "/home/kajm20/mnist/ILSVRC/Annotations/CLS-LOC/val"  

# 2. Define transformations for EfficientNet input
imagenet_transform = transforms.Compose([
    transforms.Resize(256),  
    transforms.CenterCrop(224),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

# 3. Load the synset mapping
synset_mapping_path = "/home/kajm20/mnist/ILSVRC/LOC_synset_mapping.txt"
wordnet_to_imagenet = {}

with open(synset_mapping_path) as f:
    for idx, line in enumerate(f.readlines()):
        wordnet_id, _ = line.split(' ', 1)
        wordnet_to_imagenet[wordnet_id] = idx  

# 4. Define the custom dataset class
class ImageNetValDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.annotation_files = sorted(os.listdir(annotation_dir))

    def __len__(self):
        return len(self.annotation_files)

    def __getitem__(self, idx):
        annotation_path = os.path.join(self.annotation_dir, self.annotation_files[idx])
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        wordnet_id = root.find("object").find("name").text  

        class_idx = wordnet_to_imagenet.get(wordnet_id, -1)  
        image_filename = root.find("filename").text + ".JPEG"
        image_path = os.path.join(self.image_dir, image_filename)

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, class_idx

# 5. Initialize the dataset and dataloader
imagenet_val_dataset = ImageNetValDataset(IMAGE_DIR, ANNOTATION_DIR, transform=imagenet_transform)
imagenet_val_loader = DataLoader(imagenet_val_dataset, batch_size=1, shuffle=False, num_workers=4)

# 6. Define the model (EfficientNet-B0 with pre-trained weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(weights='DEFAULT')  
model.to(device)
model.eval()  

# 7. Dictionaries to store activations
activation_shapes = {}  # Stores activation shapes
activation_values = {}  # Stores activation values

# 8. Hook function to store activations for **ALL** layers
def hook_fn(module, input, output):
    layer_name = f"{module.__class__.__name__} ({id(module)})" 
    activation_shapes[layer_name] = output.shape  # Store shape
    activation_values[layer_name] = output.detach().cpu()  # Store values (moved to CPU)

# 9. Recursively register hooks for **all** relevant layers
for name, layer in model.named_modules():
    layer.register_forward_hook(hook_fn)

# 10. Print the forward methods of all layers
def print_forward_methods(model):
    print("\n--- Forward Methods of All Layers ---\n")
    for name, layer in model.named_modules():
        if hasattr(layer, "forward"):  # Ensure the layer has a forward method
            try:
                forward_code = inspect.getsource(layer.forward)
                print(f"\n--- {name} ({layer.__class__.__name__}) ---\n")
                print(forward_code)
            except Exception as e:
                print(f"\n--- {name} ({layer.__class__.__name__}) ---\n")
                print(f"Could not retrieve forward method: {e}")

# Call the function to print forward methods
print_forward_methods(model)

# 11. Define the evaluation function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():  
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            activation_shapes.clear()  
            activation_values.clear()  
            
            print(f"\nRaw Image Tensor Shape (Before Normalization): {images.shape}")  
            
            outputs = model(images)  

            _, predicted = torch.max(outputs, 1)  
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # Print activation shapes
            print("\n--- Layer Activation Shapes ---")
            for layer, shape in activation_shapes.items():
                print(f"{layer}: {shape}")

            break  

    accuracy = (correct / total) * 100
    return accuracy

# 12. Evaluate the model on ImageNet validation set
accuracy = evaluate_model(model, imagenet_val_loader)
print(f"EfficientNet-B0 Top-1 Accuracy on ImageNet: {accuracy:.2f}%")



--- Forward Methods of All Layers ---


---  (EfficientNet) ---

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


--- features (Sequential) ---

    def forward(self, input):
        for module in self:
            input = module(input)
        return input


--- features.0 (Conv2dNormActivation) ---

    def forward(self, input):
        for module in self:
            input = module(input)
        return input


--- features.0.0 (Conv2d) ---

    def forward(self, input: Tensor) -> Tensor:
        return self._conv_forward(input, self.weight, self.bias)


--- features.0.1 (BatchNorm2d) ---

    def forward(self, input: Tensor) -> Tensor:
        self._check_input_dim(input)

        # exponential_average_factor is set to self.momentum
        # (when it is available) only so that it gets updated
        # in ONNX graph when this node is exported to ONNX.
        if self.momentum is None:
            exponential_average_factor = 0.0
        else:
   

  0%|          | 0/50000 [00:00<?, ?it/s]


Raw Image Tensor Shape (Before Normalization): torch.Size([1, 3, 224, 224])

--- Layer Activation Shapes ---
Conv2d (139326499128336): torch.Size([1, 32, 112, 112])
BatchNorm2d (139326559913920): torch.Size([1, 32, 112, 112])
SiLU (139326516731472): torch.Size([1, 32, 112, 112])
Conv2dNormActivation (139326516723792): torch.Size([1, 32, 112, 112])
Conv2d (139326485821584): torch.Size([1, 32, 112, 112])
BatchNorm2d (139326559913648): torch.Size([1, 32, 112, 112])
SiLU (139326516730272): torch.Size([1, 32, 112, 112])
Conv2dNormActivation (139326516731712): torch.Size([1, 32, 112, 112])
AdaptiveAvgPool2d (139326516730512): torch.Size([1, 32, 1, 1])
Conv2d (139326485822544): torch.Size([1, 8, 1, 1])
SiLU (139326516730992): torch.Size([1, 8, 1, 1])
Conv2d (139326485809104): torch.Size([1, 32, 1, 1])
Sigmoid (139326443186096): torch.Size([1, 32, 1, 1])
SqueezeExcitation (139326554502128): torch.Size([1, 32, 112, 112])
Conv2d (139326485822224): torch.Size([1, 16, 112, 112])
BatchNorm2d (1393

  0%|          | 0/50000 [00:00<?, ?it/s]

EfficientNet-B0 Top-1 Accuracy on ImageNet: 100.00%


In [61]:
import os
import torch
import torch.nn as nn
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm

# 1. Define paths for ImageNet validation images and annotations
IMAGE_DIR = "/home/kajm20/mnist/ILSVRC/Data/CLS-LOC/val"  
ANNOTATION_DIR = "/home/kajm20/mnist/ILSVRC/Annotations/CLS-LOC/val"  

# 2. Define transformations for EfficientNet input
imagenet_transform = transforms.Compose([
    transforms.Resize(256),  
    transforms.CenterCrop(224),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

# 3. Load the synset mapping
synset_mapping_path = "/home/kajm20/mnist/ILSVRC/LOC_synset_mapping.txt"
wordnet_to_imagenet = {}

with open(synset_mapping_path) as f:
    for idx, line in enumerate(f.readlines()):
        wordnet_id, _ = line.split(' ', 1)
        wordnet_to_imagenet[wordnet_id] = idx  

# 4. Define the custom dataset class
class ImageNetValDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.annotation_files = sorted(os.listdir(annotation_dir))

    def __len__(self):
        return len(self.annotation_files)

    def __getitem__(self, idx):
        annotation_path = os.path.join(self.annotation_dir, self.annotation_files[idx])
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        wordnet_id = root.find("object").find("name").text  

        class_idx = wordnet_to_imagenet.get(wordnet_id, -1)  
        image_filename = root.find("filename").text + ".JPEG"
        image_path = os.path.join(self.image_dir, image_filename)

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, class_idx

# 5. Initialize the dataset and dataloader
imagenet_val_dataset = ImageNetValDataset(IMAGE_DIR, ANNOTATION_DIR, transform=imagenet_transform)
imagenet_val_loader = DataLoader(imagenet_val_dataset, batch_size=1, shuffle=False, num_workers=4)

# 6. Define the model (EfficientNet-B0 with pre-trained weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(weights='DEFAULT')  
model.to(device)
model.eval()  

# 7. Dictionaries to store activations
activation_shapes = {}  # Stores activation shapes
activation_values = {}  # Stores activation values

# 8. Hook function to store activations for **ALL** layers
def hook_fn(module, input, output):
    layer_name = f"{module.__class__.__name__} ({id(module)})" 
    activation_shapes[layer_name] = output.shape  # Store shape
    activation_values[layer_name] = output.detach().cpu()  # Store values (moved to CPU)

# 9. Recursively register hooks for **all** relevant layers, including `AdaptiveAvgPool2d`
for name, layer in model.named_modules():
    layer.register_forward_hook(hook_fn)

# 10. Define the evaluation function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():  
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            activation_shapes.clear()  
            activation_values.clear()  
            
            print(f"\nRaw Image Tensor Shape (Before Normalization): {images.shape}")  
            
            outputs = model(images)  

            _, predicted = torch.max(outputs, 1)  
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # Print activation shapes
            print("\n--- Layer Activation Shapes ---")
            for layer, shape in activation_shapes.items():
                print(f"{layer}: {shape}")

            break  

    accuracy = (correct / total) * 100
    return accuracy

# 11. Evaluate the model on ImageNet validation set
accuracy = evaluate_model(model, imagenet_val_loader)
print(f"EfficientNet-B0 Top-1 Accuracy on ImageNet: {accuracy:.2f}%")


  0%|          | 0/50000 [00:00<?, ?it/s]


Raw Image Tensor Shape (Before Normalization): torch.Size([1, 3, 224, 224])

--- Layer Activation Shapes ---
Conv2d (139326242846224): torch.Size([1, 32, 112, 112])
BatchNorm2d (139326516285232): torch.Size([1, 32, 112, 112])
SiLU (139326474465792): torch.Size([1, 32, 112, 112])
Conv2dNormActivation (139326474464352): torch.Size([1, 32, 112, 112])
Conv2d (139326242845584): torch.Size([1, 32, 112, 112])
BatchNorm2d (139326516284960): torch.Size([1, 32, 112, 112])
SiLU (139326474466512): torch.Size([1, 32, 112, 112])
Conv2dNormActivation (139326474466272): torch.Size([1, 32, 112, 112])
AdaptiveAvgPool2d (139326474466752): torch.Size([1, 32, 1, 1])
Conv2d (139326242846544): torch.Size([1, 8, 1, 1])
SiLU (139326474466992): torch.Size([1, 8, 1, 1])
Conv2d (139326242846864): torch.Size([1, 32, 1, 1])
Sigmoid (139326242829520): torch.Size([1, 32, 1, 1])
SqueezeExcitation (139326569985008): torch.Size([1, 32, 112, 112])
Conv2d (139326242847184): torch.Size([1, 16, 112, 112])
BatchNorm2d (1393

In [17]:
models.efficientnet_b0()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat